# Moments, Barriers, and Classical Control in Cirq

This notebook teaches:
- What a Cirq Moment is and how automatic packing works.
- Why classical controls (measurement-key–conditioned ops) may not reshape moments by themselves.
- Techniques to enforce or visualize phase boundaries: explicit moments, identity layers, custom barrier gates.

## 1. Imports

In [ ]:
import cirq
from typing import Sequence

## 2. Core Concepts

A `cirq.Moment` is a parallel layer of operations with no shared qubits. Cirq builds moments greedily from the *surface list* of operations. It sees only:
- Which qubits each top‑level operation touches.
- Whether an operation already declares a classical control (e.g. `X(q).with_classical_controls('m')`).

For ordinary, top‑level operations this works fine: a measurement that produces key `m` goes into some earlier moment, and any later operation that lists `m` as a classical control is necessarily placed in a strictly later moment (because at scheduling time Cirq sees the dependency).

The problematic case is when you wrap a classically controlled gate that consumes a measurement key (e.g. `X(q).with_classical_controls('m')`) inside the `_decompose_` of a custom gate.

What goes wrong:
- At scheduling time the outer custom gate looks like one opaque operation touching its given qubits and has *no visible classical controls* yet.
- Cirq places that opaque gate as early as qubit conflicts allow (potentially right after the entangling steps), because it doesn’t know there are internal classical dependencies.
- Later, when the circuit is printed or transformed and the custom gate is decomposed, the internal sequence (measurements followed by their controlled corrections) is revealed. But the moment boundaries are already fixed. Both the internal measurements and their dependent corrections now appear out of order.

We will illustrate this issue and techniques to fix it below with the quantum teleportation algorithm. First, we will define a custom gate that will use classical controls in it decomposition. These classical dependencies will be hidden from the Cirq default scheduler.

In [ ]:
# --- Feed-forward corrections (decomposes to classically controlled X/Z) ---
class MessageCorrection(cirq.Gate):
    def __init__(self, b1: str, b2: str):
        self.b1 = b1; self.b2 = b2
    def _num_qubits_(self): return 1
    def _decompose_(self, qs):
        (bob,) = qs
        yield cirq.Z(bob).with_classical_controls(self.b1)
        yield cirq.X(bob).with_classical_controls(self.b2)
    def _circuit_diagram_info_(self, args): return f'Z^{self.b1}X^{self.b2}'


Next, we will bring back some familiar custom gates from the custom gate notebook. These gates don't have internal classical controls, so they won't cause problems and are just here to help us with the teleportation algorithm.

In [ ]:
# --- Helper: Bell pair preparation ---
class BellPairGate(cirq.Gate):
    def _num_qubits_(self): return 2
    def _decompose_(self, qs):
        c, t = qs
        yield cirq.H(c)
        yield cirq.CNOT(c, t)
    def _circuit_diagram_info_(self, args): return 'BP','BP'
BP = BellPairGate()

# --- Helper: Bell-basis prep between message and Alice ---
class MessagePrepGate(cirq.Gate):
    def _num_qubits_(self): return 2
    def _decompose_(self, qs):
        m, a = qs
        yield cirq.CNOT(m, a)
        yield cirq.H(m)
    def _circuit_diagram_info_(self, args): return 'ME_M','ME_A'
MPrep = MessagePrepGate()

Next, we will define the three different ways of putting the algorithm together, one with automatic packing, one with using the barrier gate, and one with explicit moments.

1. Auto-packed (flat op list, no barrier)  
   - We hand Cirq a simple linear list of operations.  
   - Cirq greedily groups them into `Moment`s based only on qubit conflicts it sees at the surface.  
   - If classical dependencies are hidden inside custom gate decompositions, their internal classical dependencies aren't considered during packing, which can result in erroneous gate ordering.
   - Pros: Minimal code, maximum automatic parallelism.
   - Cons: Can cause incorrect order of gates due to internal classical dependencies not being considered during scheduling.

2. Auto-packed with a barrier gate (visual / structural separator)  
   - We insert a custom `BarrierGate` that decomposes to no operations, but renders a vertical separator symbol on every qubit line.  
   - Because the barrier is an opaque multi‑qubit operation, Cirq places it as its own moment, forcing all subsequent operations into strictly later moments.  
   - Pros: Keeps code simple while guaranteeing a visible layer break; preserves default parallelism elsewhere.  
   - Cons: Adds an artificial artifact; downstream transforms not respecting barriers might still reorder unless they treat unknown multi‑qubit ops conservatively. This is not the officially supported technique for dealing with scheduling concerns.

3. Explicit moments (manual phase construction)  
   - We build the circuit by providing an ordered list of `cirq.Moment` objects, each with exactly the operations we want concurrent.
   - This fixes the moment structure up front: measurements reside in an earlier explicit moment; each classical correction is in its own later moment.
   - Pros: Total control; unambiguous pedagogical staging; prevents accidental merging caused by hidden dependencies. This is the officially supported way address scheduling concerns.
   - Cons: Slightly more verbose; you must manually ensure you didn’t miss parallelism you wanted. No visual indication in the printed circuit.

In [ ]:
# --- Optional full-qubit barrier (visual layer separator) ---
class BarrierGate(cirq.Gate):
    def __init__(self, num_qubits: int): self._n = num_qubits
    def _num_qubits_(self): return self._n
    def _decompose_(self, qs): return []  # no-op
    def _circuit_diagram_info_(self, args):
        return cirq.CircuitDiagramInfo(wire_symbols=('│',) * self._n)

# Teleportation ops as a flat list (auto packing may merge corrections upward if not separated)
def teleport_ops_with_optional_barrier(use_barrier: bool) -> Sequence[cirq.Operation]:
    alice = cirq.NamedQubit('alice')
    bob = cirq.NamedQubit('bob')
    msg = cirq.NamedQubit('msg')
    ops: Sequence[cirq.Operation] = []
    ops.append(BP.on(alice, bob))
    ops.append(cirq.rx(0.7)(msg))
    ops.append(MPrep.on(msg, alice))
    ops.append(cirq.measure(msg, key='b1'))
    ops.append(cirq.measure(alice, key='b2'))
    if use_barrier:
        ops.append(BarrierGate(3).on(msg, alice, bob))
    ops.append(MessageCorrection('b1','b2').on(bob))
    ops.append(cirq.measure(bob, key='result'))
    return ops

# Explicit-moment construction for guaranteed boundaries
def teleport_explicit_moments() -> cirq.Circuit:
    alice = cirq.NamedQubit('alice')
    bob = cirq.NamedQubit('bob')
    msg = cirq.NamedQubit('msg')
    return cirq.Circuit([
        cirq.Moment([cirq.H(alice)]),
        cirq.Moment([cirq.CNOT(alice, bob), cirq.rx(0.7)(msg)]),
        cirq.Moment([cirq.CNOT(msg, alice)]),
        cirq.Moment([cirq.H(msg)]),
        cirq.Moment([cirq.measure(msg, key='b1'), cirq.measure(alice, key='b2')]),
        cirq.Moment([cirq.Z(bob).with_classical_controls('b1')]),
        cirq.Moment([cirq.X(bob).with_classical_controls('b2')]),
        cirq.Moment([cirq.measure(bob, key='result')]),
    ])

## 3. Auto-Packed vs. Barrier vs. Explicit Moments
We construct three circuits:
1. Auto-packed (no barrier): may merge feed-forward corrections earlier.
2. Auto-packed with barrier: forces a visible layer boundary.
3. Explicit moments: manually defined phases, independent of greedy packing.

Below we construct and print all three variants so you can compare how the corrections land relative to the measurements in each style.

In [ ]:
auto_no_barrier = cirq.Circuit(teleport_ops_with_optional_barrier(False))
auto_with_barrier = cirq.Circuit(teleport_ops_with_optional_barrier(True))
explicit = teleport_explicit_moments()

print("Auto-packed (no barrier):")
print(auto_no_barrier)
print("\nAuto-packed (with barrier):")
print(auto_with_barrier)
print("\nExplicit moments:")
print(explicit)

## 5. Simulation
We simulate the explicit-moment circuit to sample Bob's teleported state $\ket{ψ} = Rx(0.7)\ket{0}$. Expected probabilities: $P(0) ≈ \cos^2{0.35} ≈ 0.88$, $P(1) ≈ \sin^2{0.35} ≈ 0.12$.

In [ ]:
import math

sim = cirq.Simulator()
result = sim.run(auto_with_barrier, repetitions=1000)
print("Bob's teleported state measurement histogram:")
print(result.histogram(key='result'))
# Expected probabilities
p0 = math.cos(0.35)**2
p1 = math.sin(0.35)**2
print(f"Expected P(0)≈{p0:.3f}, P(1)≈{p1:.3f}")